In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime

### Read in the data

df = pd.read_csv('../Capstone/csv/trip_cleaned.csv')

In [3]:
dff = df[['pickup_datetime','dropoff_datetime','fare_amount', 'trip_time', 'trip_distance',
          'pickup_taxizone_id', 'dropoff_taxizone_id']]
dff['pickup_datetime'] = pd.to_datetime(dff['pickup_datetime'])
dff['dropoff_datetime'] = pd.to_datetime(dff['dropoff_datetime'])
dff['pickup_datetime_interval'] = dff['pickup_datetime'].dt.round('15min').dt.time
dff['dropoff_datetime_interval'] = dff['dropoff_datetime'].dt.round('15min').dt.time
dff.head()

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value i

,pickup_datetime,dropoff_datetime,fare_amount,trip_time,trip_distance,pickup_taxizone_id,dropoff_taxizone_id,pickup_datetime_interval,dropoff_datetime_interval
0,2013-06-24 18:55:00,2013-06-24 19:01:00,6.0,6.0,0.97,148.0,79.0,19:00:00,19:00:00
1,2013-06-24 19:12:00,2013-06-24 19:30:00,14.5,18.0,3.04,164.0,143.0,19:15:00,19:30:00
2,2013-06-24 19:45:00,2013-06-24 20:10:00,17.5,25.0,4.17,142.0,79.0,19:45:00,20:15:00
3,2013-06-24 20:17:00,2013-06-24 20:28:00,10.5,11.0,2.35,234.0,141.0,20:15:00,20:30:00
4,2013-06-24 20:30:00,2013-06-24 20:35:00,6.0,5.0,1.24,141.0,263.0,20:30:00,20:30:00


### Round time to 15 minutes intervals & index

In [5]:
interval_index_table = pd.read_csv('../Capstone/csv/interval_index_table_0.csv')
conversion_15m = dict(zip(interval_index_table.interval, interval_index_table.index_15m))

In [8]:
dff['pickup_datetime_index'] = [conversion_15m[str(t)] for t in dff['pickup_datetime_interval']]
dff['dropoff_datetime_index'] = [conversion_15m[str(t)] for t in dff['dropoff_datetime_interval']]
dff.head()

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [12]:
dff.to_csv('../Capstone/csv/trip_time_index.csv', index=False)

### fare_amount: src, dst, t

In [31]:
fare_amount_agg = dff.groupby(['pickup_datetime_index', 'pickup_datetime_interval','pickup_taxizone_id', 
                               'dropoff_taxizone_id'], as_index=False)['fare_amount'].agg(['mean', 'median', 'count']).reset_index()
fare_amount_agg.head()

,pickup_datetime_index,pickup_datetime_interval,pickup_taxizone_id,dropoff_taxizone_id,mean,median,count
0,0,00:00:00,2.0,2.0,10.500,10.50,2
1,0,00:00:00,3.0,69.0,24.500,24.50,1
2,0,00:00:00,4.0,4.0,8.475,4.50,20
3,0,00:00:00,4.0,7.0,22.000,20.75,6
4,0,00:00:00,4.0,13.0,13.625,13.50,8


In [32]:
fare_amount_agg.shape

(755364, 7)

In [33]:
fare_amount_agg.to_csv('../Capstone/csv/fare_amount_scr_dst_t.csv', index=False)

### trip_distance: src, dst

In [17]:
trip_distance_agg = dff.groupby(['pickup_taxizone_id', 'pickup_datetime_id'], as_index=False)['trip_distance'].agg(['mean', 'median', 'count']).reset_index()
trip_distance_agg.head()

,pickup_taxizone_id,dropoff_taxizone_id,mean,median,count
0,1.0,1.0,2.732745,0.70,51
1,1.0,43.0,0.500000,0.50,1
2,1.0,50.0,3.030000,3.03,1
3,1.0,70.0,24.700000,24.70,1
4,1.0,79.0,15.570000,15.57,1


In [19]:
trip_distance_agg.shape

(28231, 5)

In [30]:
trip_distance_agg.to_csv('../Capstone/csv/trip_distance_scr_dst.csv', index=False)

### trip_time: src, dst, t

In [26]:
trip_time_agg = dff.groupby(['pickup_datetime_index', 'pickup_datetime_interval','pickup_taxizone_id', 'dropoff_taxizone_id'], 
                            as_index=False)['trip_time'].agg(['mean', 'median', 'count']).reset_index()
trip_time_agg.head()

,pickup_datetime_index,pickup_datetime_interval,pickup_taxizone_id,dropoff_taxizone_id,mean,median,count
0,0,00:00:00,2.0,2.0,11.091667,11.091667,2
1,0,00:00:00,3.0,69.0,17.866667,17.866667,1
2,0,00:00:00,4.0,4.0,7.935000,3.625000,20
3,0,00:00:00,4.0,7.0,20.333333,19.000000,6
4,0,00:00:00,4.0,13.0,10.604167,11.325000,8


In [27]:
trip_time_agg.shape

(755364, 7)

In [29]:
trip_time_agg.to_csv('../Capstone/csv/trip_time_scr_dst_t.csv', index=False)